In [47]:
import pandas as pd
import re

In [2]:
dems = pd.read_csv("dem_candidates.csv")

In [19]:
brookings = pd.read_csv("brookings.csv")

In [28]:
dems.head()

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Biden Endorsed?,Warren Endorsed?,Sanders Endorsed?,Our Revolution Endorsed?,Justice Dems Endorsed?,PCCC Endorsed?,Indivisible Endorsed?,WFP Endorsed?,VoteVets Endorsed?,No Labels Support?
0,Anthony White (Alabama),AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Christopher Countryman,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Doug ""New Blue"" Smith",AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,James C. Fields,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sue Bell Cobb,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
brookings.head()

,Unnamed: 0,Candidate.First.Name,Candidate.Last.Name,Candidate.State,Candidate.District,Candidate.Party,Incumbent,Freshman.Member,Candidate.Website.URL,Candidate.Gender,...,Republican,Trump.Mention,Obama.Mention,Sanders.Mention,Clinton.Mention,Special.Counsel.Mention,Travel.Ban.Mention,SinglePayer,Primary.Outcome,Primary.Runoff.Outcome
0,1,CAROL,HAFNER,AK,1,Democrat,NaN,NaN,WWW.CAROLHAFNER.COM,Female,...,0,NEGATIVE MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN
1,2,CHRISTOPHER,CUMINGS,AK,1,Democrat,NaN,NaN,https://ccc4congress.com/,Male,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN
2,3,DIMITRI,SHEIN,AK,1,Democrat,NaN,NaN,WWW.DIMITRIFORCONGRESS.COM,Male,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN
3,4,ALYSE,GALVIN,AK,1,Democrat,NaN,NaN,WWW.ALYSE4ALASKA.COM,Female,...,0,NEGATIVE MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
7,8,LIZETTA,MCCONNELL,AL,1,Democrat,NaN,NaN,https://lizzettaforcongress.com/,Female,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN


In [32]:
len(brookings), len(dems)

(973, 811)

In [20]:
brookings = brookings[brookings["Candidate.Party"] == "Democrat"]
brookings = brookings[brookings["Incumbency"] == 0]

In [30]:
len(brookings), len(dems)

(973, 811)

In [26]:
brookings.describe()

,Unnamed: 0,Candidate.District,Incumbency,Female,Democrat,Republican
count,973.000000,973.000000,973.0,973.000000,973.0,973.0
mean,1147.210689,8.892086,0.0,0.329908,1.0,0.0
std,623.668112,10.005845,0.0,0.470421,0.0,0.0
min,1.000000,0.000000,0.0,0.000000,1.0,0.0
25%,642.000000,2.000000,0.0,0.000000,1.0,0.0
50%,1152.000000,5.000000,0.0,0.000000,1.0,0.0
75%,1661.000000,12.000000,0.0,1.000000,1.0,0.0
max,2277.000000,50.000000,0.0,1.000000,1.0,0.0


In [35]:
brookings[brookings["Candidate.State"] == "AL"]

,Unnamed: 0,Candidate.First.Name,Candidate.Last.Name,Candidate.State,Candidate.District,Candidate.Party,Incumbent,Freshman.Member,Candidate.Website.URL,Candidate.Gender,...,Republican,Trump.Mention,Obama.Mention,Sanders.Mention,Clinton.Mention,Special.Counsel.Mention,Travel.Ban.Mention,SinglePayer,Primary.Outcome,Primary.Runoff.Outcome
7,8,LIZETTA,MCCONNELL,AL,1,Democrat,NaN,NaN,https://lizzettaforcongress.com/,Female,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN
8,9,ROBERT,KENNEDY,AL,1,Democrat,NaN,NaN,http://www.kennedy4alabama.org/,Male,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
10,11,AUDRI,WILLIAMS,AL,2,Democrat,NaN,NaN,https://www.audriscottwilliams.com/,Female,...,0,NEGATIVE MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN
11,12,TABITHA,ISNER,AL,2,Democrat,NaN,NaN,http://tabithaisner.com/,Female,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Winner,NaN
17,18,ADIA,WINFREY,AL,3,Democrat,NaN,NaN,https://www.drdiaforcongress.com/,Female,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN
18,19,MALLORY,HAGAN,AL,3,Democrat,NaN,NaN,https://www.haganforhouse.com/,Female,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
20,21,RICK,NEIGHBORS,AL,4,Democrat,NaN,NaN,http://rickneighborsforcongress.com/,Male,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN
21,22,LEE,AUMAN,AL,4,Democrat,NaN,NaN,https://www.aumanforcongress.com/,Male,...,0,NEUTRAL MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
24,25,PETER,JOFFRION,AL,5,Democrat,NaN,NaN,https://www.peterjoffrion.com/,Male,...,0,NEGATIVE MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
27,28,DANNER,KLINE,AL,6,Democrat,NaN,NaN,https://www.votedannerkline.com/,Male,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN


In [36]:
dems[dems["State"] == "AL"]

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Biden Endorsed?,Warren Endorsed?,Sanders Endorsed?,Our Revolution Endorsed?,Justice Dems Endorsed?,PCCC Endorsed?,Indivisible Endorsed?,WFP Endorsed?,VoteVets Endorsed?,No Labels Support?
0,Anthony White (Alabama),AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Christopher Countryman,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Doug ""New Blue"" Smith",AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,James C. Fields,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Sue Bell Cobb,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Walt Maddox,AL,Governor of Alabama,Governor,Regular,6/5/18,Advanced,NaN,On the Ballot,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Lizzetta Hill McConnell,AL,U.S. House Alabama District 1,Representative,Regular,6/5/18,Lost,NaN,NaN,-30.680000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Robert Kennedy Jr.,AL,U.S. House Alabama District 1,Representative,Regular,6/5/18,Advanced,NaN,On the Ballot,-30.680000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Audri Scott Williams,AL,U.S. House Alabama District 2,Representative,Regular,6/5/18,Lost,NaN,NaN,-33.080002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Tabitha Isner,AL,U.S. House Alabama District 2,Representative,Regular,6/5/18,Advanced,NaN,On the Ballot,-33.080002,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
dems["Candidate"][0:30]

0     Anthony White (Alabama)
1      Christopher Countryman
2       Doug "New Blue" Smith
3             James C. Fields
4               Sue Bell Cobb
5                 Walt Maddox
6     Lizzetta Hill McConnell
7          Robert Kennedy Jr.
8        Audri Scott Williams
9               Tabitha Isner
10     Adia McClellan Winfrey
11              Mallory Hagan
12                  Lee Auman
13             Rick Neighbors
14             Peter Joffrion
15               Danner Kline
16            Jared Henderson
17            Leticia Sanders
18              Chintan Desai
19              Clarke Tucker
20                 Gwen Combs
21           Jonathan Dunkley
22               Paul Spencer
23              Joshua Mahony
24              Hayden Shamel
25          Brianna Westbrook
26            Hiral Tipirneni
27            Akinyemi Agbede
28     Albert Caesar Mezzetti
29            Amanda Renteria
Name: Candidate, dtype: object

In [49]:
def extract_first_last(name):
    name = re.sub(r'\(.*?\)', '', name)
    name = re.sub(r'\".*?\"', '', name)
    parts = name.split()

    if len(parts) >= 2:
        return parts[0] + ' ' + parts[-1]
    elif len(parts) == 1:
        return parts[0]
    else:
        return ''

dems['Candidate'] = dems['Candidate'].apply(extract_first_last)

dems['Candidate']

0               Anthony White
1      Christopher Countryman
2                  Doug Smith
3                James Fields
4                    Sue Cobb
                ...          
806            Talley Sergent
807           Janice Hagerman
808                Paul Davis
809             Richard Ojeda
810              Shirley Love
Name: Candidate, Length: 811, dtype: object

In [52]:
dems['Candidate'] = dems['Candidate'].str.upper()

dems[['Candidate First Name', 'Candidate Last Name']] = dems['Candidate'].str.split(' ', n = 1, expand=True)
dems

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Sanders Endorsed?,Our Revolution Endorsed?,Justice Dems Endorsed?,PCCC Endorsed?,Indivisible Endorsed?,WFP Endorsed?,VoteVets Endorsed?,No Labels Support?,Candidate First Name,Candidate Last Name
0,ANTHONY WHITE,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ANTHONY,WHITE
1,CHRISTOPHER COUNTRYMAN,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CHRISTOPHER,COUNTRYMAN
2,DOUG SMITH,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,DOUG,SMITH
3,JAMES FIELDS,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,JAMES,FIELDS
4,SUE COBB,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SUE,COBB
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
806,TALLEY SERGENT,WV,U.S. House West Virginia District 2,Representative,Regular,5/8/18,Advanced,NaN,On the Ballot,-35.330002,...,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,TALLEY,SERGENT
807,JANICE HAGERMAN,WV,U.S. House West Virginia District 3,Representative,Regular,5/8/18,Lost,NaN,NaN,-47.480000,...,NaN,NaN,NaN,NaN,NaN,No,No,NaN,JANICE,HAGERMAN
808,PAUL DAVIS,WV,U.S. House West Virginia District 3,Representative,Regular,5/8/18,Lost,NaN,NaN,-47.480000,...,NaN,NaN,NaN,NaN,NaN,No,No,NaN,PAUL,DAVIS
809,RICHARD OJEDA,WV,U.S. House West Virginia District 3,Representative,Regular,5/8/18,Advanced,NaN,On the Ballot,-47.480000,...,NaN,NaN,NaN,NaN,NaN,Yes,Yes,NaN,RICHARD,OJEDA


In [55]:
brookings['Candidate.First.Name'] = brookings['Candidate.First.Name'].str.upper()
brookings['Candidate.Last.Name'] = brookings['Candidate.Last.Name'].str.upper()

In [56]:
dems_candidates = set(dems['Candidate First Name'] + ' ' + dems['Candidate Last Name'])
brookings_candidates = set(brookings['Candidate.First.Name'] + ' ' + brookings['Candidate.Last.Name'])

missing_in_brookings = dems_candidates - brookings_candidates

missing_in_dems = brookings_candidates - dems_candidates

print(f"Candidates in 'dems' but not in 'brookings': {missing_in_brookings}")
print(f"Candidates in 'brookings' but not in 'dems': {missing_in_dems}")

Candidates in 'dems' but not in 'brookings': {'BILL HAAS', 'STACEY ABRAMS', 'BEN JEALOUS', 'GEOFF YOUNG', 'MICHAEL JR.', 'ADAM COTE', 'RONALD DINICOLA', 'SUE ZWAHLEN', 'ARDEN ANDERSEN', 'DOUG SMITH', 'MICHAEL SHELLENBERGER', 'DONNA DION', 'GRADY YARBROUGH', 'JOHN JR.', 'BRIANNA WESTBROOK', 'JOHN ROBERSON', 'FLYNN JR.', 'PETER DILL', 'JAMES FIELDS', 'CARY KENNEDY', 'DREW EDMONDSON', 'MARK DION', 'STEVE LOUGH', 'JACK JR.', 'DANIEL BURLEIGH', 'RUSHERN III', 'JOHN CHIANG', 'STACEY EVANS', 'J.B. PRITZKER', 'MARTIN OLSEN', "RONALD O'DONNELL", 'LIZZETTA MCCONNELL', 'FRANK JR.', 'VANESSA WARD', 'JEFF APODACA', 'MARGE DOYLE', 'ROBERT MARSHALL', 'ROBERT GRIFFIS', 'MICHAEL JOHNSTON', 'JOSEPH SCHIAVONI', 'MARGUERITE WILLIS', 'CHRIS GIUNCHIGLIANI', 'JOSHUA MAHONY', 'DUWAYNE GREGORY', 'LETICIA SANDERS', 'BETSY SWEET', 'NATHAN KLEINMAN', 'FRED HUBBELL', 'JIMMY JONES', 'JARED HENDERSON', 'SID ZELLER', 'LAURA KELLY', 'MICHAEL SMITH', 'BOB DAIBER', 'TJ MULLOY', 'THOMAS CARES', 'LEANN JACOBSEN', 'ABDUL E

In [67]:
dems['Candidate First Name'] = dems['Candidate First Name'].str.upper()
dems['Candidate Last Name'] = dems['Candidate Last Name'].str.upper()
brookings['Candidate.First.Name'] = brookings['Candidate.First.Name'].str.upper()
brookings['Candidate.Last.Name'] = brookings['Candidate.Last.Name'].str.upper()

#outer merge (combine the datasets even if candidates not present in both
merged_df = pd.merge(dems, brookings, 
                     left_on=['Candidate First Name', 'Candidate Last Name'], 
                     right_on=['Candidate.First.Name', 'Candidate.Last.Name'], 
                     how='outer')

In [68]:
merged_df

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Republican,Trump.Mention,Obama.Mention,Sanders.Mention,Clinton.Mention,Special.Counsel.Mention,Travel.Ban.Mention,SinglePayer,Primary.Outcome,Primary.Runoff.Outcome
0,ANTHONY WHITE,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CHRISTOPHER COUNTRYMAN,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DOUG SMITH,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JAMES FIELDS,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SUE COBB,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN
1162,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NO MENTION,NO MENTION,POSITIVE MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN
1163,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
1164,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NEUTRAL MENTION,NO MENTION,NEUTRAL MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN


In [69]:
def fill_candidate(row):
    if pd.isna(row['Candidate']):
        if not pd.isna(row['Candidate.First.Name']) and not pd.isna(row['Candidate.Last.Name']):
            return row['Candidate.First.Name'] + ' ' + row['Candidate.Last.Name']
    return row['Candidate']

merged_df['Candidate'] = merged_df.apply(fill_candidate, axis=1)

def fill_first_last_names(row):
    if pd.isna(row['Candidate.First.Name']) or pd.isna(row['Candidate.Last.Name']):
        if not pd.isna(row['Candidate']):
            names = row['Candidate'].split(' ', 1)
            if len(names) == 2:
                row['Candidate.First.Name'], row['Candidate.Last.Name'] = names
            elif len(names) == 1:
                row['Candidate.First.Name'] = names[0]
                row['Candidate.Last.Name'] = ''
    return row

merged_df = merged_df.apply(fill_first_last_names, axis=1)

merged_df

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Republican,Trump.Mention,Obama.Mention,Sanders.Mention,Clinton.Mention,Special.Counsel.Mention,Travel.Ban.Mention,SinglePayer,Primary.Outcome,Primary.Runoff.Outcome
0,ANTHONY WHITE,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CHRISTOPHER COUNTRYMAN,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DOUG SMITH,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,JAMES FIELDS,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,SUE COBB,AL,Governor of Alabama,Governor,Regular,6/5/18,Lost,NaN,NaN,-28.879999,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1161,BEAU LIEGEOIS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN
1162,PAULA SWEARENGIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NO MENTION,NO MENTION,POSITIVE MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN
1163,GARY TRAUNER,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
1164,TRAVIS HELM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,NEUTRAL MENTION,NO MENTION,NEUTRAL MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN


In [70]:
merged_df.columns

Index(['Candidate', 'State', 'District', 'Office Type', 'Race Type',
       'Race Primary Election Date', 'Primary Status', 'Primary Runoff Status',
       'General Status', 'Partisan Lean', 'Primary %', 'Won Primary', 'Race',
       'Veteran?', 'LGBTQ?', 'Elected Official?', 'Self-Funder?', 'STEM?',
       'Obama Alum?', 'Party Support?', 'Emily Endorsed?',
       'Guns Sense Candidate?', 'Biden Endorsed?', 'Warren Endorsed? ',
       'Sanders Endorsed?', 'Our Revolution Endorsed?',
       'Justice Dems Endorsed?', 'PCCC Endorsed?', 'Indivisible Endorsed?',
       'WFP Endorsed?', 'VoteVets Endorsed?', 'No Labels Support?',
       'Candidate First Name', 'Candidate Last Name', 'Unnamed: 0',
       'Candidate.First.Name', 'Candidate.Last.Name', 'Candidate.State',
       'Candidate.District', 'Candidate.Party', 'Incumbent', 'Freshman.Member',
       'Candidate.Website.URL', 'Candidate.Gender', 'Listed.military.service.',
       'Education', 'Marital.Status', 'Previous.Electoral.Experien

In [71]:
#inner merged dataset (include only entries present in both datasets
inner_merged_df = pd.merge(dems, brookings, 
                           left_on=['Candidate First Name', 'Candidate Last Name'], 
                           right_on=['Candidate.First.Name', 'Candidate.Last.Name'], 
                           how='inner')

inner_merged_df

,Candidate,State,District,Office Type,Race Type,Race Primary Election Date,Primary Status,Primary Runoff Status,General Status,Partisan Lean,...,Republican,Trump.Mention,Obama.Mention,Sanders.Mention,Clinton.Mention,Special.Counsel.Mention,Travel.Ban.Mention,SinglePayer,Primary.Outcome,Primary.Runoff.Outcome
0,AUDRI WILLIAMS,AL,U.S. House Alabama District 2,Representative,Regular,6/5/18,Lost,NaN,NaN,-33.080002,...,0,NEGATIVE MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Loser,NaN
1,TABITHA ISNER,AL,U.S. House Alabama District 2,Representative,Regular,6/5/18,Advanced,NaN,On the Ballot,-33.080002,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,"Yes, candidate supports universal healthcare r...",Winner,NaN
2,ADIA WINFREY,AL,U.S. House Alabama District 3,Representative,Regular,6/5/18,Lost,NaN,NaN,-33.660000,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN
3,MALLORY HAGAN,AL,U.S. House Alabama District 3,Representative,Regular,6/5/18,Advanced,NaN,On the Ballot,-33.660000,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
4,LEE AUMAN,AL,U.S. House Alabama District 4,Representative,Regular,6/5/18,Advanced,NaN,On the Ballot,-62.480000,...,0,NEUTRAL MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,AARON SCHEINBERG,WV,U.S. House West Virginia District 2,Representative,Regular,5/8/18,Lost,NaN,NaN,-35.330002,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN
628,TALLEY SERGENT,WV,U.S. House West Virginia District 2,Representative,Regular,5/8/18,Advanced,NaN,On the Ballot,-35.330002,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
629,JANICE HAGERMAN,WV,U.S. House West Virginia District 3,Representative,Regular,5/8/18,Lost,NaN,NaN,-47.480000,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Loser,NaN
630,RICHARD OJEDA,WV,U.S. House West Virginia District 3,Representative,Regular,5/8/18,Advanced,NaN,On the Ballot,-47.480000,...,0,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NO MENTION,NaN,Winner,NaN
